# 1. 图片加载器

## 1.1 OCR
OCR，optical character recognition，光学字符识别。也就是说，它能够识别并 “读取” 嵌入在图像中的文本。

pytesseract: 是一款面向 Python 的光学字符识别（OCR）工具。

安装：

```shell
brew install tesseract
brew install tesseract-lang
pip install pytesseract
```

静态方法：

- image_to_string()
    - path: 图片路径
    - lang: 需要识别的语言，中文-"chi_sim"

In [14]:
# brew install tesseract-lang
# TesseractError: (1, 'Error opening data file /opt/homebrew/share/tessdata/chi_sim.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'chi_sim\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')
print(pytesseract.image_to_string("./images/AI_text.png", lang='chi_sim'))

目前，AMD 的远期市盈率为 48.3Xx，低于过去 5 年均值。联发科为 17.5X，

与自身历史比较处于偏高于均值的位置，但是相比于高速成长的 Al 算力增
量，仍有空间。英伟达的未来 12 个月市盈率为 34.9x，高通的未来 12 个月
的市到率为 13.9x，人台积电林来 12 个月的市副率为 18.2Xx，从历史估值百分
位来看估值合理。进一步看，我们认为本轮由 Al 引领的科技产业革命有望
带动行业估值突破历史新高。

估值，尤其是处于科技前沿的 Al 产业及公司。

因此，综上来看，我们认为在 Al 莫力秋片行业的估值可以得到较好支撑，
而基本面的增量则可以提供成长空间。

****************************************************************************************************


In [15]:
pytesseract.image_to_string("./images/AI_financing.png", lang='chi_sim')

'时间               公司             募次                                                                                 伟伍\n\n2024年9月        OpenAI           计划向投资者筹集 65 亿美元                                                            1570 亿美元\n\n2024年5月        智谱 Al            智谱 Al 获 Prosperity7 Ventures 4 亿元美元 C轮投资                                  200 亿人民币\n\n2024年5月        Scale Al           Al 数据标注公司 Scale Al 完成 10 亿美元的慕资                                     138 亿美元\n\n2024年5月    XAI         马斯克为 XAI 投入 60 亿美元的融资                           180 亿美元\n\n2024年5月    Kimi 月之暗面 蕉至5月，阿里巴巴 2024 财年合计投入 8 亿美元                  25 亿美元\n亚马逊宣布完成了对 Anthropic 〈Claude) 总计 40 亿美元的融资。2023 年             本\n\n2024年4月       Anthropic                                     -         184 亿美元\n9 月亚马逊提供了一笔 12.5 亿美元的投资，之后又追加了 27.5 亿美元\n\n2023 年11月 零一万物        完成 10 亿美元的新一轮融资，由阿里云领头                                   NA\n\n2023 年10 月 Anthropic       谷歌领投了 Anthropic 的5亿美元融资，此后又追加了 20 亿美元的投资 ”NA\n\n2023 年4月       OpenAI   

In [16]:
pytesseract.image_to_string("./images/AI_PE.png", lang='chi_sim')

'一一英伟达\n\n主要Al 算力秋片公司市盈率\n一一超成半导体 。 一一英特尔\n\n图表 28\n120\n100\n80\n60\n40\n20\n\n60-Vc0C\n\n90-c0C\n\n50-Vc0C\n\nZT-85Z0C\n\n60-5C0C\n\n90-5C0C\n\n50-5C0C\n\nZT-CZOC\n\n60-CCZ0C\n\n90-CZ0C\n\n50-CCZ0C\n\nZT-TZOC\n\n60-TZ0Z\n\n90-TZ0Z\n\n50-TZ0C\n\nZT-0Z0C\n\n60-0C0C\n\n90-0C0C\n\n50-0C0C\n\nZT-6T0C\n\n60-6T0<\n\n90-6T0<\n\n50-6T0<\n\nZT-8T0Z\n\n60-8T0<\n\n90-8T0<\n\nFactSet、浦银国际\n\n'

## 1.2 使用LLM(多模态)识别图片

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_ollama import ChatOllama
from langchain_community.document_loaders.base import BaseLoader
from dataclasses import dataclass
import base64
from PIL import Image

@dataclass
class RagImageLoader(BaseLoader):
    image_path: str = ""
    model: str = "qwen2.5vl:latest"
    prompt: str = """你是一个高级视觉语言模型,具备分析和理解各种类型图片的能力。请根据提供的图片内容执行以下特定的任务：

1. **全文字图片**：如果图片完全由文字组成，请提取并返回所有可见的文字内容。

2. **表格图片**：如果图片包含了一个或多个表格，请将这些表格转换为Markdown格式，并确保行列结构准确无误。注意保持原始数据的精确性和完整性。

3. **图表（如柱状图、饼图等）**：
   - 用简洁明了的文字描述图表的主要内容和趋势。
   - 包括但不限于图表的类型、各部分代表的意义及其数值或比例关系。
   - 如果可能的话，指出任何显著的趋势或异常值。

4. **插图或图标**：如果图片是用于传达概念或想法的插图或图标，请解释图像试图传达的核心信息或概念。如果存在具体的数据或事实，请一并指出。

5. **复杂混合内容**：如果图片包含了以上多种类型的信息，请分别识别并报告每种类型的内容。例如，先描述文字内容，然后转换表格，最后解释图表或插图。

6. **其他类型**：对于不属于上述类别的图片，请提供尽可能详细的描述，包括但不限于图片的主题、背景信息、潜在含义以及任何值得注意的细节。

请基于上述指南对提供的图片进行分析，并根据图片的实际内容采取相应的处理方式。如果有任何不确定之处，尽量给出合理的推测和解释。

现在，请开始分析下述图片："""

    @staticmethod
    def get_image_type(image_path: str) -> bool:
        try:
            # 使用Image.open()方法打开图像文件
            with Image.open(image_path) as img:
                # 通过img.format属性获取图像格式
                return img.format
        except IOError:
            # 如果无法打开文件，可能是无效路径或非图像文件
            print("无法打开图像文件，请检查路径是否正确或文件是否损坏")
            return None
        
    @staticmethod
    def image_to_base64(image_path: str) -> (str, str):
        with open(image_path, "rb") as image_file:
            # # 读取文件的前几个字节以识别MIME类型
            # file_head = image_file.read(8)
            
            # # 根据文件头信息判断MIME类型
            # if file_head.startswith(b'\xff\xd8'):
            #     mime_type = 'image/jpeg'
            # elif file_head.startswith(b'\x89PNG\r\n\x1a\n'):
            #     mime_type = 'image/png'
            # elif file_head.startswith(b'GIF87a') or file_head.startswith(b'GIF89a'):
            #     mime_type = 'image/gif'
            # else:
            #     mime_type = 'application/octet-stream'  # 默认类型
            
            # # 回到文件开头
            # image_file.seek(0)
            
            # 编码图片数据为base64
            base64_data = base64.b64encode(image_file.read()).decode('utf-8')

        mime_type = OllamaMultiLLMImageLoader.get_image_type(image_path)
        return mime_type, base64_data
    
    def lazy_load(self):
        if self.image_path == "" or self.image_path is None:
            raise Exception("image_path 不能为空")

        prompt = ChatPromptTemplate(
            [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": self.prompt},
                        {
                            "type": "image",
                            "source_type": "base64",
                            "mime_type": "{image_mime_type}",
                            "data": "{image_data}",
                        },
                    ],
                },
            ]
        )
        mime_type, base64_data = self.image_to_base64(image_path=self.image_path)
        llm = ChatOllama(model=self.model)
        
        chain = prompt | llm
        response = chain.invoke(
            {
                "image_data": base64_data,
                "image_mime_type": mime_type,
            }
        )
        # response: [AIMessage]
        doc = Document(
            page_content=response.content, 
            metadata={
                **(response.response_metadata or {}),
                "image_path": self.image_path
            }
        )
        return [doc]
        
